In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import tensorflow as tf

# to make sure the graph is refresh
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

/Users/jean/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# load data: digits 5 to 9, but still label with 0 to 4, 
# because TensorFlow expects label's integers from 0 to n_classes-1.
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

X_train2_full = mnist.train.images[mnist.train.labels >= 5]
y_train2_full = mnist.train.labels[mnist.train.labels >= 5] - 5
X_valid2_full = mnist.validation.images[mnist.validation.labels >= 5]
y_valid2_full = mnist.validation.labels[mnist.validation.labels >= 5] - 5
X_test2 = mnist.test.images[mnist.test.labels >= 5]
y_test2 = mnist.test.labels[mnist.test.labels >= 5] - 5

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# we want to keep only 100 instances per class in the training set 
# and let's keep only 30 instances per class in the validation set
# tesing set is already loaded above
def sample_n_instances_per_class(X, y, n=100):
    Xs, ys = [], []
    for label in np.unique(y):
        idx = (y == label)
        Xc = X[idx][:n]
        yc = y[idx][:n]
        Xs.append(Xc)
        ys.append(yc)
    return np.concatenate(Xs), np.concatenate(ys)

X_train2, y_train2 = sample_n_instances_per_class(X_train2_full, y_train2_full, n=100)
X_valid2, y_valid2 = sample_n_instances_per_class(X_valid2_full, y_valid2_full, n=30)

In [11]:
# 3-1: Softmax Only
reset_graph()
restore_saver = tf.train.import_meta_graph("./saved_model/Team35_HW2.ckpt.meta")

# 取得要transfer的圖 get graph for 1~5 layers (transfer layers) 
X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")
loss = tf.get_default_graph().get_tensor_by_name("loss:0")
Y_probability = tf.get_default_graph().get_tensor_by_name("Y_probability:0")
logists = Y_probability.op.inputs[0]
accuracy = tf.get_default_graph().get_tensor_by_name("accuracy:0")

# create new training layers
output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope="logits")
# define new training steps
learning_rate = 0.01
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam2")
training_op = optimizer.minimize(loss,var_list=output_layer_vars)

init = tf.global_variables_initializer()



In [ ]:
def train(X_train, y_train, X_validate, y_validate, train_op, epoch_bound, stop_threshold, batch_size, testing=False):

    global saver
    
    early_stop = 0
    winner_accuracy = 0.0
    winner_precision = 0.0
    winner_recall = 0.0
    
    for epoch in range(epoch_bound):

        # randomize training set
        indices_training = np.random.permutation(X_train.shape[0])
        X_train, y_train = X_train[indices_training,:], y_train[indices_training,:]

        # split training set into multiple mini-batches and start training
        total_batches = int(X_train.shape[0] / batch_size)
        for batch in range(total_batches):
            if batch == total_batches - 1:
                sess.run(train_op, feed_dict={x: X_train[batch*batch_size:], y: y_train[batch*batch_size:], mode:'TRAIN'})
            else:
                sess.run(train_op, feed_dict={x: X_train[batch*batch_size : (batch+1)*batch_size], y: y_train[batch*batch_size : (batch+1)*batch_size], mode:'TRAIN'})

        # compute validation accuracy
        cur_accuracy, cur_precision, cur_recall = evaluate(X_validate, y_validate)

        # If the accuracy rate does not increase for many times, it will early stop epochs-loop 
        if winner_accuracy < cur_accuracy:
            early_stop = 0
            winner_accuracy = cur_accuracy
            winner_precision = cur_precision
            winner_recall = cur_recall
            # save best model in testing phase
            if testing == True:
                save_path = saver.save(sess, "./saved_model/Team35_HW3_1.ckpt.ckpt")
        else:
            early_stop += 1
        if early_stop == stop_threshold:
            break

    return winner_accuracy, winner_precision, winner_recall

In [ ]:
batch_size = 512
epoch_bound = 1000
stop_threshold = 20

with tf.Session() as sess:
    
    init.run()
    # 取得tenfor的數值 restore value of transfer layers
    restore_saver = (sess, "./saved_model/Team35_HW2")

    # 初始新的layer initialize value for softmax layer
    session.run(output_layer_vars.initializer)
    train(X_train2, y_train2, x_valid2, y_valid2, training_op, epoch_bound, stop_threshold,batch)